<a href="https://colab.research.google.com/github/RenatodaCostaSantos/Machine-Learning---Lessons/blob/main/Supervised%20ML/Decision%20trees%20and%20random%20forests/Lesson_1_Introduction_to_decision_trees_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Foundations of decision trees

Decision trees are another type of supervised machine learning. Some of its advantages are that it is easier to visualize than other algorithms and convenient to present the results to a non-technical audience. 

In this course, we will use the sklearn library from python, which uses the classification and regression trees (CART) algorithm. Classification refers to predicting categorical outcomes (binary or multi-class), while the word regression refers to numerical outcomes.

In this lesson, we will focus on building a simplified version of regression and classification trees from scratch using a small dataset. It will help us understand the algorithms and appreciate the work done by the sklearn library.

# Breaking down a decision tree

Suppose we want to answer the following question:

- Can your son enter the stadium to watch a soccer game?

We need to answer a couple of other questions before answering this one. Typing each of them would be overwhelming. Decision trees provide an easy and convenient way to visualize the path to the final answer:

![decision tree](https://raw.githubusercontent.com/RenatodaCostaSantos/Machine-Learning---Lessons/main/Supervised%20ML/Decision%20trees%20and%20random%20forests/images/decision_tree.png)

The brown boxes in the figure above are called **nodes**, and they always contain a threshold used to split the data into two groups depending if the answer evaluates to True or False. The green ones are called **leaves**. The leaves are nodes that do not perform any splits on the data. In other words, that's where the algorithm terminates.

The nodes between the root and the leaf nodes are called **internal nodes**. They always have a node above them and always perform a split. The number of internal nodes depends on the complexity of the data provided.

In order to answer the question proposed at the beginning of this lesson, we generated a decision with three **levels**. It contains one root node, two internal, and four leaf nodes. It is much easier to visualize the steps followed to find the final answer than to write down all the questions involved in this procedure.


## Nodes classification

There is a classification devoted only to the nodes of a decision tree. A node divided into two subsequent nodes is called a **parent node**. The nodes generated by a parent node are called **child nodes**. This classification infers:

- The root node is not derived from any other node, making it an exclusive parent node.

- Internal nodes are both parent and child nodes.

- The leaves are exclusively child nodes.

## Branches

The arrows in a decision tree are called **branches**. The ones to the left are branches generated when the threshold evaluates True, while the ones to the right are generated when it evaluates False. The figure below breaks down the first image of this lesson making explicitly the nomenclature we just introduced:

![classification](https://raw.githubusercontent.com/RenatodaCostaSantos/Machine-Learning---Lessons/main/Supervised%20ML/Decision%20trees%20and%20random%20forests/images/classification_DT.png)


## Thresholds

As we saw in the figure above, thresholds work as if statement in a dataset, splitting the data that satisfies a condition from the rest. 

Let's see how to implement thresholds using a toy dataset. The dataset contains information about costumers that were allowed or denied entrance in a casino:


In [1]:
import pandas as pd

from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import toy dataset
casino = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Decision trees/toy_df.csv')

In [3]:
casino.head()

,banned,money_in_pocket,adult,entrance
0,False,65,True,Allowed
1,False,35,True,Denied
2,False,40,False,Denied
3,False,60,False,Denied
4,True,60,True,Denied


In [4]:
casino.shape

(10, 4)

Suppose one of the thresholds used in a decision tree is used to separate costumers that were allowed in the casino from the ones that were denied. This threshold can be implemented as:

In [5]:
# Implementing threshold that splits allowed and denied costumers
child1_true = casino[casino['entrance'] == 'Allowed']
child1_false = casino[casino['entrance'] == 'Denied']

In [6]:
child1_true.head()

,banned,money_in_pocket,adult,entrance
0,False,65,True,Allowed
6,False,60,True,Allowed
7,False,60,True,Allowed
8,False,65,True,Allowed


In [7]:
child1_false.head()

,banned,money_in_pocket,adult,entrance
1,False,35,True,Denied
2,False,40,False,Denied
3,False,60,False,Denied
4,True,60,True,Denied
5,True,35,True,Denied


Now that we have split the data into costumers that were allowed and denied in the casino, we can implement a secondary split using, for example, if the client in a client that was allowed in the casino had more or less than 50 dollars in the pocket:

In [8]:
# Implementing a secondary split using money in the pocket
child2_true = child1_true[child1_true['money_in_pocket'] > 50]
child2_false = child1_true[child1_true['money_in_pocket'] <= 50]

In [9]:
child2_true.head()

,banned,money_in_pocket,adult,entrance
0,False,65,True,Allowed
6,False,60,True,Allowed
7,False,60,True,Allowed
8,False,65,True,Allowed


In [10]:
child2_false.head()

,banned,money_in_pocket,adult,entrance


That's a way to implement a thresholds in python. A decision tree, however, would keep splitting the data until a **pure leaf** node is achieved. For example, if we wanted to know if a costumer was banned from the casino, we would have to keep splitting the data until achieving a leaf node containing only costumers that were banned and another with costumers that were not banned.

## How do we choose threholds?

It is clear now how thresholds are applied in decision trees, however, how are they chosen? Just randomly? Is there a way to optimize a choice for a threshold?

Let's try to answer how the thresholds are chosen. The optimization part will be explained later on. 

To start with, let's consider a random sample of our casino dataset that contains only 10 observations. To make it simpler we will only consider the banned and money_in_pocket columns and try to predict the entrance status of the costumer:




In [11]:
# Create a sample
sample = casino[['banned','money_in_pocket','entrance']]

In [12]:
sample

,banned,money_in_pocket,entrance
0,False,65,Allowed
1,False,35,Denied
2,False,40,Denied
3,False,60,Denied
4,True,60,Denied
5,True,35,Denied
6,False,60,Allowed
7,False,60,Allowed
8,False,65,Allowed
9,False,40,Denied



In order to find a list of candidates for the thresholds, **we need to examine every feature**. The target variable, entrance, does not need to be split.

Let's start examining the money_in_pocket column and explain how to find a list of candidates for thresholds for this column. We will follow the steps below:

1 - The first step is to take the money_in_pocket column and exclude duplicates:

In [13]:
# Exclude duplicates on money_in_pocket feature
sample['money_in_pocket'].drop_duplicates().sort_values()

1    35
2    40
3    60
0    65
Name: money_in_pocket, dtype: int64

2 - Find the average of each pair of consecutive numbers in the list above to obtain a list of possible thresholds. In the example above we would have:

2.1 - (35+40)/2 = 37.5,

2.2 - (40+60)/2 = 50,

2.3 - (60+65)/2 = 62.5.

Thus, the list of possible thresholds would be $[37.5, 50, 62.5]$.

For the banned column we repeat the same procedure. You may wonder how is that possible given that it is a column with categorical values True and False. Pandas considers boolean values as 0 for False and 1 for True values. In fact, let's compute the average of a True and a False value:

In [14]:
# Average of boolean values
( True + False )/2

0.5

In this case, the list of values with possible thresholds for the banned column is given by only one value $[0.5]$. We can apply this procedure to any binary categorical feature. However, one has to convert the non-boolean binary values as Positive/Negative, approved/disapproved, etc, to 0 and 1. In the next lesson, we will cover how to deal with multi-class categorical features.

# Optimal thresholds

We now know how to obtain a list of possible values for a threshold for a feature. The next step is to learn how we choose the values that will be optimal, in other words, the values that will lead to pure leaves with a minimal amount of steps.

Given a decision tree, there are a few different criterions that sklearn can use to achieve an optimal value for the thresholds. Check the criterion variable in the sklearn class for [regression](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) and [classification](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) problems if you are interested to know all of them. We will focus on two methods for each class in this introductory lesson.

For regression problems we will explain:

- The Mean Squared Error (MSE),

- The Mean Absolute Error (MAE).

For classification problems we will explain:

- The Gini impurity,

- Entropy & Gini Information Gain.

These methods rely on how effectively the split separates values of the target variable. Even though the thresholds are conditions imposed on the features, it will be optimal if it splits the values/categories for the target variable faster than their peers. 

## The Mean Squared Error for regression trees

Let's start with the MSE. First, we need to know what we mean by errors. The definition is simple. Take the mean of the target variable and compute the distance of each value of this variable to the mean. These distances are called the **errors**. The mean squared error is the mean of all errors squared. Mathematically:

$$
\text{MSE} =  \frac{\sum^{n}_{i=1}(\bar{x} - x_i)^2}{n},
$$

where, $\bar{x}$, is the mean value for the target variable, $x_i$ are the single values for the target variables and $n$ is the total number of observations in the dataset.

The MSE is a criterion that can be used to find an optimal value of thresholds. The steps we need to perform to find the optimal threshold value are the following:

1 - Consider a value for the threshold. It will split the data into two sets, one that evaluates True and another that evaluates False. 

2 - Compute the MSE for both datasets.

3 - Add both MSEs.

4 - Repeat steps 1-3 for every threshold.

5 - Compare the values of the MSEs and choose the threshold with the lowest MSE value.

Let's apply this criterion to a new toy dataset containing the weekly salary of workers of a casino and the number of hours worked. We already computed the thresholds for the worked_hours feature and found the list $[28, 37.5, 46.5, 55]$. Let's compute the MSE for those values and compare than to find the optimal threshold.








In [15]:
# Read the toy dataset
employees = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Decision trees/toy_casino_salary.csv')

The calculation will be repetitive so we will define a function that calculate the MSE value given a threshold.

In [16]:
def calculate_mse(data, threshold, feature, target):
  # Step1 - Separate dataset using the threshold for feature variable 
  threshold1_True = data[data[feature] <= threshold]
  threshold1_False = data[data[feature] > threshold] 

  print(f'For threshold = {threshold}, we have:')
  print()

  # Step 2 - Calculate means for target variable
  mean1 = threshold1_True[target].mean()
  print(f'The mean for the target of the True branch was {mean1:.2f}.')
  mean2 = threshold1_False[target].mean()
  print(f'The mean for the target of the False branch was {mean2:.2f}.')


  # Caluculate MSE for the target variable 
  mse1 = sum((threshold1_True[target] - mean1)**2)/threshold1_True.shape[0]
  print(f'The MSE for the True branch was {mse1:.2f}.')
  mse2 = sum((threshold1_False[target] - mean2)**2)/threshold1_False.shape[0]
  print(f'The MSE for the False branch was {mse2:.2f}.')

  print()

  # Step 3
  print(f'The total MSE for threshold = {threshold} was {mse1 + mse2:.2f}.')

In [17]:
# MSE for threshold = 28
calculate_mse(employees, 28, 'worked_hours', 'salary')
print()

# MSE for threshold = 37.5
calculate_mse(employees, 37.5, 'worked_hours', 'salary')
print()

# MSE for threshold = 46.5
calculate_mse(employees, 46.5, 'worked_hours', 'salary')
print()

# MSE for threshold = 55
calculate_mse(employees, 55, 'worked_hours', 'salary')

For threshold = 28, we have:

The mean for the target of the True branch was 598.00.
The mean for the target of the False branch was 1063.75.
The MSE for the True branch was 0.00.
The MSE for the False branch was 68340.19.

The total MSE for threshold = 28 was 68340.19.

For threshold = 37.5, we have:

The mean for the target of the True branch was 644.00.
The mean for the target of the False branch was 1188.33.
The MSE for the True branch was 2116.00.
The MSE for the False branch was 29036.22.

The total MSE for threshold = 37.5 was 31152.22.

For threshold = 46.5, we have:

The mean for the target of the True branch was 774.33.
The mean for the target of the False branch was 1265.00.
The MSE for the True branch was 35384.22.
The MSE for the False branch was 25921.00.

The total MSE for threshold = 46.5 was 61305.22.

For threshold = 55, we have:

The mean for the target of the True branch was 856.75.
The mean for the target of the False branch was 1426.00.
The MSE for the True branch

The lowest value for the MSE was provided by the threshold value of 37.5. It is the optimal value for the threshold in our example, according to the MSE criterion.

## Mean Absolute Error for regression trees

The calculation procedure to obtain the MAE for decision trees is very similar to the MSE. The key differences are:

- We use the **median** instead of the mean to calculate the errors,

- We use the absolute value of the errors to calculate the MAE.

Mathematically, the MAE is defined as:
$$
\text{MAE} = \frac{\sum^{n}_{i=1}|x_i - \tilde{x}|}{n},
$$
where, $\tilde{x}$ is the median value for the target variable, $x_i$ are single values for the target variable and $n$ is the number of observations.

The steps for obtaining the optimal threshold using the MAE criterion are the same as for the MSE. Let's use it in our toy dataset to find the best value for the threshold according to the MAE criterion.




In [18]:
def calculate_mae(data, threshold, feature, target):
  # Step1 - Separate dataset using the threshold for feature variable 
  threshold1_True = data[data[feature] <= threshold]
  threshold1_False = data[data[feature] > threshold] 

  print(f'For threshold = {threshold}, we have:')
  print()

  # Step 2 - Calculate medians for target variable
  median1 = threshold1_True[target].median()
  print(f'The median for the target of the True branch was {median1:.2f}.')
  median2 = threshold1_False[target].median()
  print(f'The median for the target of the False branch was {median2:.2f}.')


  # Caluculate MAE for the target variable 
  mae1 = sum(abs(threshold1_True[target] - median1))/threshold1_True.shape[0]
  print(f'The MSE for the True branch was {mae1:.2f}.')
  mae2 = sum(abs(threshold1_False[target] - median2))/threshold1_False.shape[0]
  print(f'The MSE for the False branch was {mae2:.2f}.')

  print()

  # Step 3
  print(f'The total MSE for threshold = {threshold} was {mae1 + mae2:.2f}.')

In [19]:
# MAE for threshold = 28
calculate_mae(employees, 28, 'worked_hours', 'salary')
print()

# MAE for threshold = 37.5
calculate_mae(employees, 37.5, 'worked_hours', 'salary')
print()

# MAE for threshold = 46.5
calculate_mae(employees, 46.5, 'worked_hours', 'salary')
print()

# MAE for threshold = 55
calculate_mae(employees, 55, 'worked_hours', 'salary')

For threshold = 28, we have:

The median for the target of the True branch was 598.00.
The median for the target of the False branch was 1069.50.
The MSE for the True branch was 0.00.
The MSE for the False branch was 201.25.

The total MSE for threshold = 28 was 201.25.

For threshold = 37.5, we have:

The median for the target of the True branch was 644.00.
The median for the target of the False branch was 1104.00.
The MSE for the True branch was 46.00.
The MSE for the False branch was 130.33.

The total MSE for threshold = 37.5 was 176.33.

For threshold = 46.5, we have:

The median for the target of the True branch was 690.00.
The median for the target of the False branch was 1265.00.
The MSE for the True branch was 145.67.
The MSE for the False branch was 161.00.

The total MSE for threshold = 46.5 was 306.67.

For threshold = 55, we have:

The median for the target of the True branch was 862.50.
The median for the target of the False branch was 1426.00.
The MSE for the True branch

The lowest value for the MAE was provided by the threshold value of 37.5. It is the optimal value for the threshold in our example, according to the MAE criterion.

## The Gini Impurity for classification trees

Suppose we randomly choose a label for the target variable of a given observation. What is the probability that our choice is wrong? In other words, what is the probability of misclassifying the target value for that observation?

The Gini impurity provides the answer to the question above. It is defined as:

$$
\text{Gini} = 1 - \sum_{i=1}^n (p_i)^2
$$
where, $n$ is the number of observations and $p_i$ is the probability of an observation belonging to a class $i$.

Let's consider some examples to develop an intuition about this definition.

1 - Suppose we have a box with 10 blue balls. What is the probability of mislabelling the color of a ball in this case? Well, they are all blue, so it should be 0. Let's compute the Gini impurity value:
$$
\text{Gini} = 1 - \bigg(\frac{10}{10}\bigg)^2 = 0.
$$
Bam!

2 - Suppose the box has 5 blue balls and 5 green ones. If we pick a random ball without looking, what is the probability of misclassifying the color of the ball we picked? Well, there are only two colors, so if we choose green, we have a 50% chance of getting it wrong. Let's compute the Gini impurity once more:
$$
\text{Gini} = 1 - \bigg(\frac{5}{10}\bigg)^2 +\bigg(\frac{5}{10}\bigg)^2  = \frac{1}{2} = 50\%.
$$
Double BAM!

3 - Suppose there are 5 different colors, where two of each ball are of a given color. If we repeat the experiment above, what is the probability of misclassifying the ball we picked? Well, we have a 20% chance to classify it correctly, which means an 80% chance to misclassify it. Let's compute the Gini impurity:
$$
\text{Gini} = 1 - 5\times\bigg(\frac{2}{10}\bigg)^2  = \frac{4}{5} = 80\%.
$$
Triple BAM!

We see that the smaller the value for the Gini impurity, the smaller the chance we are misclassifying a label.

For classification trees, we need to know if we are misclassifying labels at every split. Thus, if we calculate the Gini impurity for each threshold, we can select the one that provides the lowest Gini impurity. 

In general, the split is not even, and we have to calculate the weighted Gini impurity:

$$
\text{Weighted Gini} = \bigg( \frac{\text{# observations True branch}}{\text{# observations parent}}\times \text{Gini}_{\text{true child}} + \frac{\text{# observations False branch}}{\text{# observations parent}}\times \text{Gini}_{\text{false child}}  \bigg),
$$
where the fractions multiplying the Gini values for the True and False branches are the weights. 

Let's apply it in our toy casino dataset. We already computed the thresholds and found the values $[37.5, 50, 62.5]$ as candidates for the money_in_pocket variable. The computation of the Gini impurity for each of these thresholds will be repetitive, so we will build a function that does that for us:



In [20]:
def weighted_gini(data, threshold, feature, target):
  # Step1 - Separate dataset using the threshold for feature variable 
  threshold_True = data[data[feature] <= threshold]
  threshold_False = data[data[feature] > threshold] 

  print(f'For threshold = {threshold}, we have:')
  print()

  # Calculate Gini for the true child of the target variable
  gini_true = 1 - ((threshold_True[threshold_True[target] == 'Allowed'].shape[0]/threshold_True.shape[0])**2 + (threshold_True[threshold_True[target] == 'Denied'].shape[0]/threshold_True.shape[0])**2)
  print(f'The Gini impurity for the threshold = {threshold}, in the True branch is {gini_true:.2f}.')
  # Calculate Gini for the false child of the target variable
  gini_false = 1 - ((threshold_False[threshold_False[target] == 'Allowed'].shape[0]/threshold_False.shape[0])**2 + (threshold_False[threshold_False[target] == 'Denied'].shape[0]/threshold_False.shape[0])**2)
  print(f'The Gini impurity for the threshold = {threshold}, in the False branch is {gini_false:.2f}.')

  # Calculate weights
  weight_true = threshold_True.shape[0]/data.shape[0]
  print(f'The weight for the true child branch is {weight_true:.2f}')
  weight_false = threshold_False.shape[0]/data.shape[0]
  print(f'The weight for the false child branch is {weight_false:.2f}')


  # Calculate weighted Gini
  wG = gini_true*weight_true + gini_false*weight_false

  print(f'The weighted Gini impurity for threshold = {threshold} is {wG:.2f}.')

In [21]:
# Weighted Gini impurity for threshold = 37.5
weighted_gini(casino,37.5,'money_in_pocket', 'entrance')
print()

# Weighted Gini impurity for threshold = 50
weighted_gini(casino,50,'money_in_pocket', 'entrance')
print()

# Weighted Gini impurity for threshold = 62.5
weighted_gini(casino,62.5,'money_in_pocket', 'entrance')

For threshold = 37.5, we have:

The Gini impurity for the threshold = 37.5, in the True branch is 0.00.
The Gini impurity for the threshold = 37.5, in the False branch is 0.50.
The weight for the true child branch is 0.20
The weight for the false child branch is 0.80
The weighted Gini impurity for threshold = 37.5 is 0.40.

For threshold = 50, we have:

The Gini impurity for the threshold = 50, in the True branch is 0.00.
The Gini impurity for the threshold = 50, in the False branch is 0.44.
The weight for the true child branch is 0.40
The weight for the false child branch is 0.60
The weighted Gini impurity for threshold = 50 is 0.27.

For threshold = 62.5, we have:

The Gini impurity for the threshold = 62.5, in the True branch is 0.38.
The Gini impurity for the threshold = 62.5, in the False branch is 0.00.
The weight for the true child branch is 0.80
The weight for the false child branch is 0.20
The weighted Gini impurity for threshold = 62.5 is 0.30.


The lowest value for the weighted Gini impurity was provided by the threshold value of 50. It is the optimal value for the threshold in our example, according to the Gini impurity criterion.

# Entropy and Information Gain for decision trees

The last criterion we will cover in this lesson is also the hardest. The concept of **entropy** we will introduce here comes from [information theory](https://en.wikipedia.org/wiki/Entropy_(information_theory)). It measures the average level of surprise of an outcome for a given variable. We will not explain the derivation of the equation, but if you are interested in it and the intuition behind its derivation, check out this [statquest](https://www.youtube.com/watch?v=YtebGVx-Fxw&ab_channel=StatQuestwithJoshStarmer).

The level of surprise is directly related to how heterogeneous a group of observations is. To make it clear, suppose you have a box with 100 chickens and 100 cats (sounds like a bad idea, but let's imagine it). If we decided to pick one animal out of the box, we would be surprised to have selected a cat since we know the box contains a heterogeneous group of chickens and cats. In other words, the more heterogenous the group of observations is, the more surprised we become with the result. On the other hand, if we have a box with only one cat and 99 chickens and we picked a chicken out of the box, we would not be surprised by the outcome. In other words, the more homogeneous the group is, the smaller the average level of surprise.

The ultimate goal of decision trees is to select homogenous groups in the end, so the entropy can be used as a criterion when deciding the best threshold in a given split of the data.  

The entropy is defined as:
$$
S = - \sum^{n}_{i=1} p_i \times \log_2(p_i),
$$
where, 
$$
p_i = \frac{\text{# of observations of class $i$}}{\text{total # of observations}}
$$
is the probability of obtaining a specific class from a group of observations. Note that if $p_i = 0$, the $\log(p_i)$ is not defined. In this case, by convention we evaluate the logarithm to 0.

Let's check how it works in an example. Suppose we have a box with 10 blue balls and pick a ball out of the box. The probability of picking a blue ball from the box is 1. In this case:
$$
S = - 1\times \log_2(1) = 0.
$$
Since the entropy measures how heterogenous a group of observations is, this result makes sense. Let's consider a less ideal scenario.

Suppose the example with 100 cats and 100 chickens we introduced before. In this case, both classes, cats and chickens, have $p_i = 1/2$. In this case:
$$
S = -\frac{1}{2} \times \log_2 (1/2) - \frac{1}{2} \times \log_2 (1/2) = 1
$$
The entropy increases, meaning the average level of surprise increases, which also means the group is more heterogenous than the example with the blue balls above. In fact, $S=1$ is the maximum entropy for a binary set of observations. 

The maximum value for the entropy changes if we have more labels in our set of observations. For example, a group with 4 different labels will have maximum entropy equal to 2. If the number of labels is 8, the maximum entropy is equal to 3, and so on. 

At the end of the day, we are interested in the threshold that performs the best split in a decision tree. The entropy alone does not contain the information we need. The quantity we will use is called **information gain**, which is just the weighted difference of the entropy between the parent node and the child branches. Mathematically:
$$
\text{IG} = S_{\text{parent}} - \bigg(\frac{\text{# observations true split}}{\text{parent # observations}} \times S_{\text{true child}} + \frac{\text{# observations false split}}{\text{parent # observations}} \times S_{\text{false child}}\bigg)  
$$

Let's apply the information gain to decide the best value for the threshold option of our casino example. Remember that we computed the possible thresholds and found a list, $[37.5,50,62.5]$. We want to decide which of these thresholds will provide an optimal split. Since the calculation will be repetitive we will build a function that execute the calculations:

In [22]:
from math import log

In [32]:
def IG(data, threshold, feature, target):
  # Separate dataset using the threshold for feature variable 
  threshold_True = data[data[feature] <= threshold]
  threshold_False = data[data[feature] > threshold] 

  print(f'For threshold = {threshold}, we have:')
  print()

  # Calculate probabilities and log probabilities
  prob_true_allowed = threshold_True[threshold_True[target] == 'Allowed'].shape[0]/threshold_True.shape[0]
  if  prob_true_allowed == 0:
    log_true_allowed = 0
  else:
    log_true_allowed = log(prob_true_allowed,2)

  prob_true_denied = threshold_True[threshold_True[target] == 'Denied'].shape[0]/threshold_True.shape[0]
  if prob_true_denied == 0:
    log_true_denied = 0
  else:
    log_true_denied = log(prob_true_denied,2)

  prob_false_allowed = threshold_False[threshold_False[target] == 'Allowed'].shape[0]/threshold_False.shape[0]
  if prob_false_allowed == 0:
    log_false_allowed = 0
  else:
    log_false_allowed = log(prob_false_allowed,2)

  prob_false_denied = threshold_False[threshold_False[target] == 'Denied'].shape[0]/threshold_False.shape[0]
  if  prob_false_denied == 0:
    log_false_denied = 0
  else:
    log_false_denied = log(prob_false_denied,2)

  prob_parent_allowed = data[data[target] == 'Allowed'].shape[0]/data.shape[0]
  prob_parent_denied = data[data[target] == 'Denied'].shape[0]/data.shape[0]

  # Calculate entropy for the true child of the target variable
  entropy_true = - (prob_true_allowed*log_true_allowed + prob_true_denied*log_true_denied)
  print(f'The entropy for the threshold = {threshold}, in the True branch is {entropy_true:.2f}.')
  # Calculate entropy for the false child of the target variable
  entropy_false =  - (prob_false_allowed*log_false_allowed + prob_false_denied*log_false_denied)
  print(f'The entropy for the threshold = {threshold}, in the False branch is {entropy_false:.2f}.')

  # Calculate weights
  weight_true = threshold_True.shape[0]/data.shape[0]
  print(f'The weight for the true child branch is {weight_true:.2f}')
  weight_false = threshold_False.shape[0]/data.shape[0]
  print(f'The weight for the false child branch is {weight_false:.2f}')

  # Calculate parent entropy
  entropy_parent = - (prob_parent_allowed*log(prob_parent_allowed,2) + prob_parent_denied*log(prob_parent_denied,2))

  # Calculate information gain
  IG =  entropy_parent - (entropy_true*weight_true + entropy_false*weight_false)
  print()
  print(f'The information gain for threshold = {threshold} is {IG:.2f}.')

In [33]:
# Information gain for threshold = 37.5
IG(casino,37.5,'money_in_pocket', 'entrance')
print()

# Information gain for threshold = 50
IG(casino,50,'money_in_pocket', 'entrance')
print()

# Information gain for threshold = 62.5
IG(casino,62.5,'money_in_pocket', 'entrance')

For threshold = 37.5, we have:

The entropy for the threshold = 37.5, in the True branch is -0.00.
The entropy for the threshold = 37.5, in the False branch is 1.00.
The weight for the true child branch is 0.20
The weight for the false child branch is 0.80

The information gain for threshold = 37.5 is 0.17.

For threshold = 50, we have:

The entropy for the threshold = 50, in the True branch is -0.00.
The entropy for the threshold = 50, in the False branch is 0.92.
The weight for the true child branch is 0.40
The weight for the false child branch is 0.60

The information gain for threshold = 50 is 0.42.

For threshold = 62.5, we have:

The entropy for the threshold = 62.5, in the True branch is 0.81.
The entropy for the threshold = 62.5, in the False branch is -0.00.
The weight for the true child branch is 0.80
The weight for the false child branch is 0.20

The information gain for threshold = 62.5 is 0.32.


Since we have binary labels at the target variable, the entropy must be between 0 and 1. That is coherent with the values above. 

The highest value for the information gain was provided by the threshold value of 50. It is the optimal value for the threshold in our example, according to the information gain criterion.

# Summary

In this lesson, we introduced the following concepts:

- The structure and definitions of every item of a decision tree.

- How to find possible threshold values for a decision tree node.

- The Mean Squared Error criterion to select thresholds optimal in a **regression** tree.

- The Mean Absolute Error criterion to select optimal thresholds in a **regression** tree.

- The Gini Impurity criterion to select optimal thresholds in a **classification** tree.

- The information gain to select optimal thresholds in a **classification** tree.